In [1]:
import pandas as pd

#dir_path = "/home/vmagent/app/recdp/dataset/outbrain_raw/sampled"
dir_path = "/home/vmagent/app/recdp/dataset/outbrain_raw/orig"

file_list = {
    'clicks': "clicks_train.csv",
    'documents_categories': "documents_categories.csv",
    'documents_entities': "documents_entities.csv",
    'documents_meta': "documents_meta.csv",
    'documents_topics': "documents_topics.csv",
    'events': "events.csv",
    'page_views': "page_views_sample.csv",
    'promoted_content': "promoted_content.csv"
}

df_list = {}

for f_name, f_path in file_list.items():
    df_list[f_name] = {"data": pd.read_csv(f"{dir_path}/{f_path}")}

/tmp/ipykernel_9957/3406387447.py:20: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  df_list[f_name] = {"data": pd.read_csv(f"{dir_path}/{f_path}")}


In [2]:
for df_name, df_info in df_list.items():
    print(f"{df_name}: {list(df_info['data'].columns)}")

clicks: ['display_id', 'ad_id', 'clicked']
documents_categories: ['document_id', 'category_id', 'confidence_level']
documents_entities: ['document_id', 'entity_id', 'confidence_level']
documents_meta: ['document_id', 'source_id', 'publisher_id', 'publish_time']
documents_topics: ['document_id', 'topic_id', 'confidence_level']
events: ['display_id', 'uuid', 'document_id', 'timestamp', 'platform', 'geo_location']
page_views: ['uuid', 'document_id', 'timestamp', 'platform', 'geo_location', 'traffic_source']
promoted_content: ['ad_id', 'document_id', 'campaign_id', 'advertiser_id']


In [3]:
# define necessary functions

def find_column_with_prefix(df, to_find_name):
    idx = 0
    rename_list = {}
    for col_name in df.columns:
        if col_name.startswith(f"{to_find_name}_"):
            rename_list[col_name] = f"{to_find_name}_{idx}"
            idx += 1
    return rename_list
            
def combine_same_columns(df):
    to_rename_columns = []
    for col_name in df.columns:
        if col_name.endswith("_x"):
            to_rename_dict = {col_name: col_name[:-2]}
            df = df.rename(columns = to_rename_dict)
        if col_name.endswith("_y"):
            if col_name[:-2] not in to_rename_columns:
                to_rename_columns.append(col_name[:-2])
    # find all columns use to_rename_columns
    to_rename_dict = {}
    for col_name in to_rename_columns:
        to_rename_dict.update(find_column_with_prefix(df, col_name))
    if(len(to_rename_dict) > 0):
        print(to_rename_dict)
    df = df.rename(columns = to_rename_dict)
    return df

def merge_tables(df_list, main_table, to_merge):
    ret_df = main_table
    for df_name, df_info in df_list.items():
        if df_name not in to_merge:
            continue
        df_indexed = df_info["data"]
        #print(df_indexed)
        with Timer(f"merge {df_name} to main_table"):
            ret_df = ret_df.merge(df_indexed, on=df_info["primary_keys"], how="outer")
            ret_df = combine_same_columns(ret_df)
            ret_df = ret_df.dropna(subset=['clicked'])
    return ret_df

def find_tables_share_same_feature(df_list, target_table, candidates):
    related_tables = {}
    for t in candidates:
        for key in df_list[t]['data'].columns:
            if key in target_table.columns:
                if t not in related_tables:
                    related_tables[t] = []
                related_tables[t].append(key)
    return related_tables

In [4]:
df_list["clicks"]["data"]

,display_id,ad_id,clicked
0,1,42337,0
1,1,139684,0
2,1,144739,1
3,1,156824,0
4,1,279295,0
...,...,...,...
87141726,16874592,186600,0
87141727,16874593,151498,1
87141728,16874593,282350,0
87141729,16874593,521828,0


In [5]:
from utils import Timer
# 1. use forward methods to find related tables
table_relations = {}
target_table = df_list["clicks"]["data"]
candidate_tables = ["documents_meta", "documents_categories", "documents_entities", "documents_topics", "events", "page_views", "promoted_content"]
with Timer("Detect relations"):
    table_relations = find_tables_share_same_feature(df_list, target_table, candidate_tables)
    print(table_relations)

with Timer("merge tables"):
    for table_name, primary_keys in table_relations.items():
        df_list[table_name]["primary_keys"] = primary_keys
    train_df = merge_tables(df_list, target_table, table_relations.keys())

train_df

{'events': ['display_id'], 'promoted_content': ['ad_id']}
Detect relations took 9.569106623530388e-05 sec
  merge events to main_table took 37.26339034503326 sec
{'document_id_y': 'document_id_0'}
  merge promoted_content to main_table took 108.16486324183643 sec
merge tables took 145.42854580213316 sec


,display_id,ad_id,clicked,uuid,document_id,timestamp,platform,geo_location,document_id_0,campaign_id,advertiser_id
0,1.0,42337.0,0.0,cb8c55702adb93,379743.0,6.100000e+01,3,US>SC>519,938164,5969,1499
1,2236.0,42337.0,0.0,da19c8b40f20ca,1784646.0,1.566500e+05,3,US>WI>669,938164,5969,1499
2,3497.0,42337.0,1.0,fc1ba1ea4f4e56,1775473.0,2.392310e+05,1,US>CA>807,938164,5969,1499
3,6361.0,42337.0,0.0,8a9becddc8250d,495833.0,4.358600e+05,1,US>CA>828,938164,5969,1499
4,7709.0,42337.0,0.0,fd73225e2607bc,1757446.0,5.301860e+05,1,US>NV>839,938164,5969,1499
...,...,...,...,...,...,...,...,...,...,...,...
87141726,16874193.0,248844.0,1.0,e3395657c416a4,2817014.0,1.123173e+09,2,US>AZ>753,1616608,5125,628
87141727,16874295.0,547068.0,0.0,d527b3be8424d7,25640.0,1.123179e+09,1,US,2813576,18908,283
87141728,16874411.0,495235.0,0.0,58b8e5302205b4,945413.0,1.123187e+09,1,US>SC>567,2338203,32743,1669
87141729,16874538.0,104841.0,0.0,9f4a1ca3124d35,2788096.0,1.123196e+09,1,US>MO>609,687101,13561,305


In [ ]:
# 2. use forward methods to find related tables
table_relations = {}
target_table = train_df
candidate_tables = ["documents_meta", "documents_categories", "documents_entities", "documents_topics", "page_views",]
with Timer("Detect relations"):
    table_relations = find_tables_share_same_feature(df_list, target_table, candidate_tables)
    print(table_relations)

with Timer("merge tables"):
    # 2. merge table based on the detected relation
    for table_name, primary_keys in table_relations.items():
        df_list[table_name]["primary_keys"] = primary_keys
    train_df = merge_tables(df_list, target_table, table_relations.keys())
train_df

{'documents_meta': ['document_id'], 'documents_categories': ['document_id'], 'documents_entities': ['document_id'], 'documents_topics': ['document_id'], 'page_views': ['uuid', 'document_id', 'timestamp', 'platform', 'geo_location']}
Detect relations took 9.964988566935062e-05 sec
  merge documents_categories to main_table took 119.98256125883199 sec
{'confidence_level_y': 'confidence_level_0'}
  merge documents_entities to main_table took 342.63554000807926 sec
  merge documents_meta to main_table took 405.4968110891059 sec


In [ ]:
train_df.dtypes

# Backup

In [19]:
# 1. do data wrangling to each table
def get_day_from_timestamp(ts_s):
    np_ts = ts_s.to_numpy()
    np_day = (np_ts / 1000 / 3600 / 24).astype(int)
    return pd.Series(np_day)

def index_table(df, groupby_key):
    list_keys = [i for i in df.columns if i != groupby_key]
    return df.groupby(groupby_key).agg({i: lambda x: list(x) for i in list_keys})

# TODO: using incremental method to fill publisher_id for every documents_meta entry

# TODO: create new feature for geo info

df_list['events']['day'] = get_day_from_timestamp(df_list['events']['timestamp'])
df_list['page_views']['day'] = get_day_from_timestamp(df_list['page_views']['timestamp'])

df_list['documents_categories'] = index_table(df_list['documents_categories'], 'document_id')
df_list['documents_topics'] = index_table(df_list['documents_topics'], 'document_id')
df_list['documents_entities'] = index_table(df_list['documents_entities'], 'document_id')


In [6]:
# 2. merge documents files with 'document_id' as key into one documents_df

documents_df = df_list['documents_meta'].merge(df_list['documents_categories'], on="document_id", how="left")
documents_df = documents_df.merge(df_list['documents_topics'], on="document_id", how="left")
documents_df = documents_df.merge(df_list['documents_entities'], on="document_id", how="left")

In [14]:
# 3. use page_views and documents to create users_profile_df

page_views_train_df = df_list['page_views'].merge(documents_df, on='document_id', how='left')

# select columns
# fill na
# 1. group by index key
users_profile_df = index_table(page_views_train_df, "uuid")
# 2. create new features: num_views, categories, topics, entities
# noted: categories, topics, entities are tf-idfs -> TODO

In [21]:
# 4. use document_meta and page_views to add more info for events

events_df = df_list['events'].merge(df_list['documents_meta'], on="document_id", how="left")
events_df = events_df.merge(df_list['page_views'], on=["uuid", "document_id", "platform", "geo_location", "day"], how="left")

In [27]:
clicks_train_df = df_list['clicks'].merge(df_list['promoted_content'], on = 'ad_id', how='left')
clicks_train_df = clicks_train_df.merge(df_list['documents_meta'], on = 'document_id', how='left')
clicks_train_df = clicks_train_df.merge(events_df, on = 'display_id', how='left')

In [28]:
clicks_train_df

,display_id,ad_id,clicked,document_id_x,campaign_id,advertiser_id,source_id_x,publisher_id_x,publish_time_x,uuid,document_id_y,timestamp_x,platform,geo_location,day,source_id_y,publisher_id_y,publish_time_y,timestamp_y,traffic_source
0,7485900,34053,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,4060011,94860,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,6753026,367622,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2811702,64255,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,16687744,230212,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
871477,5768881,406501,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
871478,14904301,470780,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
871479,275389,221644,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
871480,3315522,262638,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
# 2. create documents meta
# 2.1 convert documents infos to use 'documents_id' as index
df_list['documents_categories'] = df_list['documents_categories'].groupby("document_id").apply(list)


documents_df = df_list['documents_meta'].merge(df_list['documents_categories'], on="document_id", how="left")
documents_df = documents_df.merge(df_list['documents_topics'], on="document_id", how="left")
documents_df = documents_df.merge(df_list['documents_entities'], on="document_id", how="left")

    documents_categories_grouped_df,
    on=F.col("document_id_doc") == F.col("documents_categories_grouped.document_id_cat"),
    how='left') \
    .join(documents_topics_grouped_df,
          on=F.col("document_id_doc") == F.col("documents_topics_grouped.document_id_top"),
          how='left') \
    .join(documents_entities_grouped_df,
          on=F.col("document_id_doc") == F.col("documents_entities_grouped.document_id_ent"),
          how='left') \
    .cache()


In [3]:
train_df[['display_id', 'day_event']]

,display_id,ad_id,clicked,document_id_x,campaign_id,advertiser_id,uuid,document_id_y,timestamp,platform,geo_location
0,7485900,34053,0,873183,4988,1973,NaN,NaN,NaN,NaN,NaN
1,4060011,94860,0,842023,8043,394,NaN,NaN,NaN,NaN,NaN
2,6753026,367622,0,1550906,29896,1650,NaN,NaN,NaN,NaN,NaN
3,2811702,64255,0,747047,8568,1077,NaN,NaN,NaN,NaN,NaN
4,16687744,230212,1,934549,19284,3283,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...
871447,5768881,406501,0,1670786,27502,2879,NaN,NaN,NaN,NaN,NaN
871448,14904301,470780,0,2319414,17402,1055,NaN,NaN,NaN,NaN,NaN
871449,275389,221644,0,1559982,5125,628,NaN,NaN,NaN,NaN,NaN
871450,3315522,262638,0,1635743,26690,1684,NaN,NaN,NaN,NaN,NaN


In [ ]:
# promoted_content process
# drop na of geo_location and platform

In [6]:
for f_name, df in df_list.items():
    print(f_name)
    display(df)

clicks


,display_id,ad_id,clicked
0,7485900,34053,0
1,4060011,94860,0
2,6753026,367622,0
3,2811702,64255,0
4,16687744,230212,1
...,...,...,...
871412,5768881,406501,0
871413,14904301,470780,0
871414,275389,221644,0
871415,3315522,262638,0


documents_categories


,document_id,category_id,confidence_level
0,2352362,1708,0.780317
1,1594739,1702,0.920000
2,1805482,1909,0.920000
3,2918616,1909,0.070000
4,2612162,1702,0.372515
...,...,...,...
54810,932786,1408,0.920000
54811,120841,1513,0.920000
54812,1115090,1702,0.033258
54813,1072518,1904,0.056042


documents_entities


...

,display_id,uuid,document_id,timestamp,platform,geo_location
0,4967935,40dfeb57f000ee,1198481,320930355,1,US>AL>698
1,8666478,82056b07568174,2272350,584950772,1,ID>30
2,20999551,9c5d03fac43aaa,1381525,1184004787,2,NL>15
3,1241053,f88f4d99844dd4,1479631,74551999,1,US>CA>862
4,14537827,35d20b3563cdfc,2305575,957265831,2,US>WI>617
...,...,...,...,...,...,...
231196,2140178,a81a8be3c30997,859508,138016907,2,US>VA>573
231197,15648428,d01a974f0c8cdf,1526957,1039456560,2,US>GA>524
231198,22777619,fcbf55cd6c90a7,2945036,1277332641,1,US>TX>623
231199,3755812,3e0870229c1773,2038568,239683133,2,US>WI>669


page_views


,uuid,document_id,timestamp,platform,geo_location,traffic_source
0,7db77ca87e4258,1786253,40516871,1,US>CO>751,1
1,5938a1ec6e3af5,1482450,49732213,1,PE>15,3
2,ad8604581c44f8,1812224,48401515,1,US>LA>622,1
3,b1727224174818,379054,31169451,2,HK>00,2
4,7e031459822b2d,95516,42031607,2,US>AZ>753,2
...,...,...,...,...,...,...
99995,dc40b2f27face0,1833998,47418035,2,US>TX>625,1
99996,117a07a5cbb4f2,838069,47145780,2,US>NY>501,2
99997,1e8fd7477f588b,234,61175061,3,US>MN>613,1
99998,7da577646a63e7,25792,53891075,1,US>MA>506,1


promoted_content


,ad_id,document_id,campaign_id,advertiser_id
0,279845,1671112,27551,3074
1,28185,668644,4204,1352
2,17829,748438,2931,1636
3,193642,1484963,22788,3543
4,195462,866412,17909,1452
...,...,...,...,...
5591,40191,602076,5664,1890
5592,326105,1751141,29116,2176
5593,36590,899623,5286,823
5594,397970,1109919,30667,2874
